# 検出

学習したモデルを使って、実際に画像から検出をします。

## 概要

学習が進めば、実際に画像を読み込んでそこから対象物を検出できるようになります（はずです）。

このノートブックでは、物体検出を体験してみます。  
検出モデルは前のステップで学習したモデル（を検出用に変換（後述））、入力は検証用画像です。  
また「検出がどれくらいうまくいっているか」を測るための、mAP（Mean Average Precision）という評価指標とその計算方法も紹介します。

## 準備

In [ ]:
import sys
sys.path.append(r'../models/research')

import os
from collections import defaultdict
from io import StringIO

import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from PIL import Image

from object_detection.utils import ops as utils_ops


In [ ]:
# This is needed to display the images.
%matplotlib inline

In [ ]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

↑ matplotlib の設定でコンフリクトが起きるのでエラーが出ますが、notebookの動作には支障ありません。無視してください。

## モデル準備

学習したモデル（グラフデータおよびチェックポイントファイル）を、変数情報を固定化して検出に特化したモデル（フローズングラフ）に変換します。

↓ `model.ckpt-2000` の箇所は、実際に回したステップ数に応じて適宜修正してください（100ステップなら `model.ckpt-100`、等）。

In [ ]:
%%time
%%sh
env PYTHONPATH=$PYTHONPATH:../models/research:../models/research/slim \
    python ../models/research/object_detection/export_inference_graph.py \
        --input_type image_tensor \
        --pipeline_config_path ./ssd_mobilenet_v1_manabiya.pbtxt \
        --trained_checkpoint_prefix ./train/model.ckpt-2000 \
        --output_directory ./frozen

↑ `Converted 199 variables to const ops.` と出力されればOK。  
　警告が出る場合がありますが無視してください。

変換に成功すると、`./frozen/` ディレクトリ以下にいくつかのファイルが生成されます。

## 変数設定

検出で使用するいくつかの変数を定義します。

In [ ]:
# フローズングラフへのパス。これが実際に物体検出に利用されるモデルです。
PATH_TO_FROZEN_GRAPH = './frozen/frozen_inference_graph.pb'

# ラベルマップファイルへのパス（Step1 で生成したもの。出力ラベル表示時に利用）
PATH_TO_LABELS = './label_map.pbtxt'

# クラス数
NUM_CLASSES = 36

# 表示画像サイズ（Jupyter Notebook 上に検出結果を表示する際にのみ使用）
DISPLAY_IMAGE_SIZE = (12*2, 8*2)

## モデルの読込

`PATH_TO_FROZEN_GRAPH` からモデルを読み込みます。TensorFlow の `tf.Graph` および `tf.GraphDef` を利用します。

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

## ラベルマップの読込

`PATH_TO_LABELS` からラベルマップを読み込みます。TF Object Detection で用意されているユーティリティ関数を利用します。

In [ ]:
# ラベルマップをそのまま読込
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
# ↑を「ディクショナリのリスト」に変換（評価時に利用）
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
# ↑をさらに「ディクショナリ（ID->（ラベルマップの1項目を表す）ディクショナリ）」に変換（検出結果表示時に利用）
category_index = label_map_util.create_category_index(categories)

## ヘルパーコード

読み込んだ画像をRGBのバイト列（3次元配列）に変換する関数を準備します。  
検出する際には画像ファイルの生データではなくバイト列にデコードしたデータを入力とする必要があり、そのためのものです。

In [ ]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

## 入力

準備された検証用の画像を読み込む準備をします。  
ここでは画像ディレクトリのパスと「input_data_id と ファイル名の対応辞書」準備しているだけです。  

In [ ]:
"""画像ディレクトリのパス"""
PATH_TO_TEST_IMAGES_DIR = './data-manabiya/width1000/image'

In [ ]:
"""input_data_id と ファイル名の対応辞書"""
input_data_id_2_input_name = {
    "2e152501-d5d9-4026-bde0-7c28c1e773e0":"540782936.273833.png",
    "41816be2-3f57-4be9-bfd3-d228a4c1ca69":"540782823.324211.png",
    "418fc9c8-3ec4-49b2-8176-a8cf5376c026":"540782586.412328.png",
    "85228d61-209f-4675-b30b-fd842b5623a1":"540782457.810163.png",
    "b257a1e9-433d-480d-a41e-ef1dbf29d67e":"540782780.456003.png",
    "b4c190e3-9599-4d7f-9d97-d23b09752fdc":"540782910.323475.png"
}

## 検出実行

`run_inference_for_single_image()` 関数は、画像とグラフ（検出用のフローズングラフ）を受け取って、推測（検出）を実行する関数です。  
長いのでかいつまんで仕様（概要）を説明します：

+ 引数：
    + `image`: 画像データ（縦×横×RGBの3次元配列）
    + `graph`: フローズングラフ（を読み込んだ `tf.Graph`）
+ 処理概要：
    1. フローズングラフから、検出結果を返すテンソル（ops）を抽出
    2. 受け取った画像データを引き渡して推測→検出結果を取得
    3. 数値を適宜修正（補正）して返す
+ 戻り値：
    + 辞書：
        + 各キーとその値：
            + `num_detections`: 検出数
            + `detection_boxes`: 検出した矩形（BoundingBox）のリスト
            + `detection_classes`: 検出したクラスのリスト
            + `detection_scores`: 検出結果のスコアのリスト
        + 各 `detection_～es` は `num_detections` 個の要素からなるリスト（numpy array）
        + 各 `detection_～es` の各`n`番目の要素は互いに対応（`n`番目の矩形<->`n`番目のクラス<->`n`番目のスコア）
        + `detection_scores` はスコアの高いものから降順で並んでいる
    

In [ ]:
def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                    'num_detections', 'detection_boxes', 'detection_scores',
                    'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                            tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                        detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                        tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                        detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict, feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
                    'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

### 検出実行（評価用）

`TEST_IMAGE_PATHS` から各画像を読み込んで検出結果を収集および表示します。

In [ ]:
# input_data_id と「出力結果の辞書」の対応辞書（準備）
output_dicts_dict = {}

In [ ]:
%%time
for (input_data_id, filename) in input_data_id_2_input_name.items():
    image_path = os.path.join(PATH_TO_TEST_IMAGES_DIR, filename)
    image_np = load_image_into_numpy_array(Image.open(image_path))
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    output_dicts_dict[input_data_id] = output_dict

    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
            image_np,
            output_dict['detection_boxes'],
            output_dict['detection_classes'],
            output_dict['detection_scores'],
            category_index,
            instance_masks=output_dict.get('detection_masks'),
            use_normalized_coordinates=True,
            line_thickness=8)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)

## 評価

検証用画像の検出結果を利用して、mAP（Mean Average Precision）を計算します。  
まずは、mAP（およびその周辺知識）の概要を説明します。

+ Precision と Recall：
    + Precision（適合率）：
        + 「正しい」と判定されたもののうち、実際に正しかったものの割合
        + Precision が高いほど、『誤検出』が少ない
    + Recall（再現率）：
        + 実際に正しいもののうち、「正しい」と判定されたものの割合
        + Recall が高いほど、『検出漏れ』が少ない
+ AP（Average Precision：平均適合率）：
    + 各 Recall に対する Precision の平均を取ったもの
+ mAP（Mean Average Precision）：
    + 各クラスごとに算出した AP の平均を取ったもの

In [ ]:
from object_detection.utils import config_util
from object_detection import evaluator as od_evaluator

In [ ]:
configs = config_util.get_configs_from_pipeline_file('./ssd_mobilenet_v1_manabiya.pbtxt')
eval_config = configs['eval_config']

In [ ]:
evaluator = od_evaluator.get_evaluators(eval_config, categories)[0]

In [ ]:
import json

In [ ]:
def bounding_box(coordinates):
    _x = 0
    _y = 1
    _xs = [t[_x] for t in coordinates]
    _ys = [t[_y] for t in coordinates]
    min_x, min_y = min(_xs), min(_ys)
    max_x, max_y = max(_xs), max(_ys)
    return (min_x, min_y, max_x, max_y)

In [ ]:
class_text_2_index = {item['name']: item['id'] for item in categories}

In [ ]:
input_data_id_2_gtbs = {}
input_data_id_2_classes = {}
for input_data_id in input_data_id_2_input_name.keys():
    annotation_json_path = os.path.join("./data-manabiya/width1000/json", "{}.json".format(input_data_id))
    json_dict = json.load(open(annotation_json_path, 'r'))
    # input_data_id = json_dict["input_data_id"]
    # assert input_data_id == json_dict["input_data_id"]
    # 画像サイズは決め打ち
    width,height = 1000.0, 750.0
    gtbs = []
    classes = []
    for annotation in json_dict["detail"]:
        flat_coordinates = annotation["data"].split(",")
        xy_coordinates = [(int(x),int(y)) for (x,y) in zip(*[iter(flat_coordinates)]*2)]
        min_x, min_y, max_x, max_y = bounding_box(xy_coordinates)
        gtbs.append([min_y / height, min_x / width, max_y / height, max_x / width])
        class_text = str(annotation["additional_data_list"][0]["choice"])
        classes.append(class_text_2_index[class_text])
    input_data_id_2_gtbs[input_data_id] = np.array(gtbs)
    input_data_id_2_classes[input_data_id] = np.array(classes)

↑警告が出ることがありますが気にしないでください。

In [ ]:
for input_data_id in output_dicts_dict.keys():
    evaluator.add_single_ground_truth_image_info(input_data_id, {
        "groundtruth_boxes": input_data_id_2_gtbs[input_data_id],
        "groundtruth_classes": input_data_id_2_classes[input_data_id]
    })
    evaluator.add_single_detected_image_info(input_data_id, output_dicts_dict[input_data_id])

↑警告が出ることがありますが気にしないでください。

In [ ]:
eval_output_dict = evaluator.evaluate()
eval_output_dict